In [8]:
'''		CREATED BY GAURAV MITRA		'''
'''		CRAWLS AND SCRAPES GOOGLE SCHOLAR WEBSITE ON ENTERING A QUERY		'''
# WARNINGS :  
# 		1)		GOOGLE SCHOLAR robots.txt SITE CLEARLY FORBIDS CRAWLERS AND SCRAPERS.
#		2)		MOST OF THE DATA IN GOOGLE SCHOLAR IS INVISIBLE TO ROBOTS AND CRAWLERS
#		3)		HENCE TOO MUCH QUERYING MAY LEAD TO TEMPORARY BAN ON YOUR IP ADDRESS

'''		LIGHTWEIGHT CRAWLER THAT CRAWLS THROUGH THE GOOGLE SCHOLAR SITE MOSTLY RESPECTING GOOGLE TERMS OF USE BUT MAY LEAD TO 
		OCCASSIONAL VIOLATION OF THE robots.txt OF GOOGLE SCHOLAR   '''

try :
	import urllib
	import urllib2
	#import BeautifulSoup as bs
	import re
	import random
	import robotparser as rp
	import htmlentitydefs
	import hashlib
	import sys
	import os
	import datetime
	import time
except ImportError:
	print 'Error : {}'.format("All Required Modules are Not Present")

HEADERS=dict()

def initialization() :
        return "http://scholar.google.com","/scholar?q="

def parse_Robot_File():
	robot=rp.RobotFileParser()
	GOOGLE_SCHOLAR_MAIN_URL,QUERY_APPEND=initialization()
	robot.set_url(GOOGLE_SCHOLAR_MAIN_URL+"/robots.txt")
	return robot.read()
	
def random_number_seedset() :
	date=datetime.datetime.now()
	time_since_epoch=time.mktime(date.timetuple()) + date.microsecond/1000000.0
	milliseond=time_since_epoch*1000
	random.seed(milliseond)

def create_fake_id() :
	random_number_seedset()
	random_string=str(random.random())
	random_string=random_string.encode('utf8')
	fake_google_id=hashlib.md5(random_string)
	fake_google_id=fake_google_id.hexdigest()
	fake_google_id=fake_google_id[:16]
	HEADERS['Cookie']="GSP=ID={}:CF=4".format(fake_google_id)
    HEADERS['User-Agent']="Mozilla/5.0"


def Google_Scholar_Query(searchQuery):
        create_fake_id(),
        GOOGLE_SCHOLAR_URL,QUERY_APPEND=initialization()
        searchQuery.replace(" ","+")
        url = GOOGLE_SCHOLAR_URL + QUERY_APPEND + searchQuery
        html=""
        try :
                request = urllib2.Request(url, headers=HEADERS)
                response = urllib2.urlopen(request)
                page_data = response.read()
                page_data = page_data.decode('utf8')
                html=page_data
        except urllib2.HTTPError : 
                print HEADERS
                print '{0} : {1}'.format("HTTP 503 SERVICE UNAVAILABLE ERROR","CHANGE USER-AGENT IN HEADERS TO FIX THE ERROR")

        link_parse = re.compile(r'<a href="(/scholar\.bib\?[^"]*)')
        links = link_parse.findall(html)
        links = [re.sub('&(%s);' % '|'.join(htmlentitydefs.name2codepoint), lambda m: chr(htmlentitydefs.name2codepoint[m.group(1)]), item) for item in links]

        output = list()
        for link in links:
                if not allowed_By_Robots(link) :
                        continue
                url = GOOGLE_SCHOLAR_URL+link
                try :
                        request = urllib2.Request(url, headers=header)
                        response = urllib2.urlopen(request)
                        bibtex = response.read()
                        bibtex = bibtex.decode('utf8')
                        output.append(bibtex)
                except urllib2.HTTPError :
                        print '{0} : {1}'.format("HTTP 503 SERVICE UNAVAILABLE ERROR","CHANGE USER-AGENT IN HEADERS TO FIX THE ERROR")
                        continue
        return output

def allowed_By_Robots(link):
	robot=parse_Robot_File()
	if (robot.can_fetch("*",link) is True) :
		return True
	else :
		return False

if __name__=="__main__" :
	print '{}'.format("You Have to Enter the Search Query and This Aplication will return a list of Citations that match your Query")
	query=raw_input("Enter the Name of the Paper or Query String : ")
	citations_list=Google_Scholar_Query(query)
	print citations_list


IndentationError: unindent does not match any outer indentation level (<ipython-input-8-4b0910b19aeb>, line 52)